In [1]:
from config import Config
from selenium import webdriver
from selenium.webdriver.common.by import By
# import Select
from selenium.webdriver.support.ui import Select
import logging
import urllib
from time import sleep



In [2]:
logging.basicConfig(level=logging.INFO)

options = webdriver.ChromeOptions()
options.add_argument(f'user-data-dir=/Users/suchattangjarukij/Library/Application Support/Google/Chrome/Profile 1')
options.add_argument('--no-sandbox')

driver = webdriver.Chrome(options=options)

In [3]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def handle_field(label, field_group):
    ret = None
    match label:
        case "Email address":
            ret = field_group.find_element(By.CSS_SELECTOR, "select").get_attribute("value")
        case "Phone country code":
            elem = field_group.find_element(By.CSS_SELECTOR, "select")
            select = Select(elem)
            select.select_by_value("Australia (+61)")
            ret = elem.get_attribute("value")
        case "Mobile phone number":
            elem = field_group.find_element(By.CSS_SELECTOR, "input")
            elem.clear()
            elem.send_keys("0420797445")
            ret = elem.get_attribute("value")
        case _:
            try:
                elem = field_group.find_element(By.CSS_SELECTOR, "input")
                elem.clear()
                elem.send_keys("1")
                ret = elem.get_attribute("value")
            except:
                pass

            try:
                elem = field_group.find_element(By.CSS_SELECTOR, "select")
                select = Select(elem)
                select.select_by_index(1)
                ret = elem.get_attribute("value")
            except:
                pass

    return ret

def fillin_modal(modal):
    try:
        fields = {}
        while True:
            mobile_fields = modal.find_elements(By.CLASS_NAME, "jobs-easy-apply-form-section__grouping")

            for field_group in mobile_fields:
                label = field_group.find_element(By.CSS_SELECTOR, "label")
                label_text = label.text.split("\n")[0].strip()
                fields[label_text] = {"value": handle_field(label_text, field_group)}

            try:
                next_button = WebDriverWait(modal, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[aria-label='Continue to next step']")))
                next_button.click()
            except:
                close_button = modal.find_element(By.CSS_SELECTOR, "button[aria-label='Dismiss']")
                close_button.click()
                discard_button = modal.find_element(By.XPATH, "//span[text()='Discard']")
                discard_button.click()
                return fields

        return fields
    except Exception as e:
        logging.warning(e)
        pass


In [13]:
params = {
    'keywords': 'data analyst',
    "location": "Melbourne, Victoria, Australia",
    "f_AL": "true",
    "distance":"10"
    }
    
driver.get("https://www.linkedin.com/jobs/search/?"+urllib.parse.urlencode(params))
driver.implicitly_wait(2)

In [17]:
# find container
job_card_container = driver.find_element(By.CSS_SELECTOR, "ul.scaffold-layout__list-container")
# find clickable title
job_cards = job_card_container.find_elements(By.CLASS_NAME, "artdeco-entity-lockup__title")



In [18]:
limit = 5
iteration = 0
results = []
while job_cards:
    result = {"index":iteration}

    iteration+=1
    if iteration > limit:
        break

    try:
        job_cards = job_card_container.find_elements(By.CLASS_NAME, "artdeco-entity-lockup__title")
        job_card = job_cards[iteration]


        # scroll to the element
        driver.execute_script("arguments[0].scrollIntoView();", job_card)
        driver.implicitly_wait(2)

        
        

        # switch to the element
        job_card.click()
        driver.implicitly_wait(5)
        sleep(5)
        logging.info(f"{iteration}:{job_card.text}")


        result = {
            **result, 
            "title": job_card.text, 
            "url": driver.current_url
            }

        

        #click easy apply
        for i in range(10):
            try:
                driver.implicitly_wait(1)
                easy_apply_button = driver.find_element(By.CLASS_NAME, "jobs-apply-button")
                easy_apply_button.click()
                
            except:
                logging.info("No easy apply button")
                continue

            finally:
                break
            


        # get the modal
        for i in range(10):
            try:
                modal = driver.find_element(By.CLASS_NAME, "jobs-easy-apply-modal")
                driver.implicitly_wait(1)

            except:
                logging.info("No modal")
                continue

            finally:
                break
       
        

        # fill in the modal
        try:
            fields = fillin_modal(modal)
            result = {**result, **fields}
        except:
            pass

        
    except Exception as e:
        logging.info("{}".format(str(e)))
        continue

    finally:
        results.append(result)
        driver.implicitly_wait(2)
    
   

  

results

INFO:root:1:SQL Data Analyst


Email address
Phone country code
Mobile phone number
found next button
found next button
How many years of work experience do you have with SQL?
Are you an australian citizen/PR
found next button
Yes
No next button


INFO:root:2:Master Data Analyst


Email address
Phone country code
Mobile phone number
found next button
found next button
How many years of work experience do you have with Oracle Database?
How many years of work experience do you have with SAP ERP?
How many years of work experience do you have with Negotiation?
Yes
No next button


INFO:root:3:Environmental Data Analyst


...

INFO:root:4:Senior Data Analyst


Email address
Phone country code
Mobile phone number
found next button
found next button
Yes
No next button


INFO:root:5:Senior Business Analyst - Data Analytics


Email address
Phone country code
Mobile phone number
found next button
found next button
Are you a Australian citizen/PR ?
How many years of experience do you have as a Data Business Analyst
No next button


[{'index': 0,
  'title': 'SQL Data Analyst',
  'url': 'https://www.linkedin.com/jobs/search/?currentJobId=3774147184&distance=10&f_AL=true&keywords=data%20analyst&location=Melbourne%2C%20Victoria%2C%20Australia',
  'Email address': {'value': 'suchat.tangjarukij@proton.me'},
  'Phone country code': {'value': 'Australia (+61)'},
  'Mobile phone number': {'value': '0420797445'},
  'How many years of work experience do you have with SQL?': {'value': '1'},
  'Are you an australian citizen/PR': {'value': 'Yes'},
  'Yes': {'value': None}},
 {'index': 1,
  'title': 'Master Data Analyst',
  'url': 'https://www.linkedin.com/jobs/search/?currentJobId=3777875903&distance=10&f_AL=true&keywords=data%20analyst&location=Melbourne%2C%20Victoria%2C%20Australia',
  'Email address': {'value': 'suchat.tangjarukij@proton.me'},
  'Phone country code': {'value': 'Australia (+61)'},
  'Mobile phone number': {'value': '0420797445'},
  'How many years of work experience do you have with Oracle Database?': {'valu

In [127]:
driver.quit()

In [25]:
results

[{'index': 0,
  'title': 'Master Data Analyst',
  'url': 'https://www.linkedin.com/jobs/search/?currentJobId=3777875903&distance=10&f_AL=true&keywords=data%20analyst&location=Melbourne%2C%20Victoria%2C%20Australia'},
 {'index': 1,
  'title': 'Senior Analytics Consultant - Azure, AWS - Snowflake - Databricks',
  'url': 'https://www.linkedin.com/jobs/search/?currentJobId=3774140560&distance=10&f_AL=true&keywords=data%20analyst&location=Melbourne%2C%20Victoria%2C%20Australia'},
 {'index': 2},
 {'index': 3},
 {'index': 4}]